# 🎬 VideoRobot — One-Click Colab Runner (Fixed)
این نوت‌بوک ریپوی VideoRobot را کلون کرده، وابستگی‌ها را نصب می‌کند، بک‌اند Flask را بالا می‌آورد، سلامت آن را بررسی می‌کند، یک درخواست نمونه برای رندر ارسال می‌کند و ویدیوی نهایی را دانلود می‌کند.

In [ ]:
# 🧠 Mount Google Drive for assets (optional)
from google.colab import drive
drive.mount('/content/drive')
print('✅ Google Drive mounted. Put assets under drive/MyDrive/VideoRobot/Assets if needed.')

In [ ]:
# 📦 Clone repository fresh
import shutil, pathlib, subprocess, sys
REPO_URL = "https://github.com/englishpodcasteasy-glitch/videorobot.git"
WORK = pathlib.Path('/content/videorobot')
shutil.rmtree(WORK, ignore_errors=True)
subprocess.run(['git', 'clone', REPO_URL, str(WORK)], check=True)
print('✅ Repo cloned at', WORK)

# اضافه کردن مسیر پروژه به sys.path برای ایمپورت‌ها
if str(WORK) not in sys.path:
    sys.path.insert(0, str(WORK))

In [ ]:
# ⚙️ Install FFmpeg and Python deps
!apt-get update -qq && apt-get install -y ffmpeg
!pip install -q -r /content/videorobot/requirements.txt

In [ ]:
# 🚀 Start backend on port 8000 and check health
import os, time, requests, subprocess, sys, logging

# تنظیم لاگ برای دیدن خروجی‌های بک‌اند در Colab
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

os.environ['VR_HOST'] = '0.0.0.0'
os.environ['VR_PORT'] = '8000'

# مسیر فایل اصلی بک‌اند شما
MAIN_SCRIPT = WORK / 'main.py'

if not MAIN_SCRIPT.exists():
    raise FileNotFoundError(f"Main script not found at {MAIN_SCRIPT}")

print(f"🚀 Starting backend server: {MAIN_SCRIPT}")
backend = subprocess.Popen([sys.executable, str(MAIN_SCRIPT)], cwd=str(WORK))

base_url = f"http://127.0.0.1:{os.environ['VR_PORT']}"
health_url = f"{base_url}/health"

print(f"⏳ Waiting for backend to become healthy at {health_url}...")

for i in range(120):
    try:
        r = requests.get(health_url, timeout=2)
        if r.ok:
            print('✅ Backend is healthy!')
            print('Response:', r.json())
            break
    except requests.exceptions.ConnectionError as e:
        # این خطا در ابتدا طبیعی است، چون سرور هنوز بالا نیامده
        pass
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        pass
    time.sleep(1)
else:
    # اگر از حلقه خارج شد، یعنی بک‌اند سالم نشده است
    print("❌ Backend did not become healthy in time.")
    print("Checking backend logs for errors...")
    stdout, stderr = backend.communicate(timeout=5)
    if stderr:
        print("--- STDERR ---")
        print(stderr.decode())
    if stdout:
        print("--- STDOUT ---")
        print(stdout.decode())
    raise SystemExit('❌ Backend failed to start.')

In [ ]:
# 📝 Create a sample manifest (preset) for the render job
import json, pathlib

# مسیر فایل‌های شما در Google Drive
DRIVE_ASSETS = pathlib.Path('/content/drive/MyDrive/VideoRobot/Assets')

# لیست فایل‌های صوتی و تصویر موجود در Drive
audio_files = list(DRIVE_ASSETS.glob('*.mp3')) + list(DRIVE_ASSETS.glob('*.wav'))
image_files = list(DRIVE_ASSETS.glob('*.jpg')) + list(DRIVE_ASSETS.glob('*.png'))

if not audio_files or not image_files:
    raise FileNotFoundError(
        "❌ No audio or image files found in Google Drive Assets folder. "
        f"Please upload them to {DRIVE_ASSETS}"
    )

# استفاده از اولین فایل‌های موجود به عنوان نمونه
audio_file = audio_files[0].name
bg_image = image_files[0].name

print(f"🎧 Using audio file: {audio_file}")
print(f"🖼️ Using background image: {bg_image}")

# ساخت یک manifest (preset) نمونه
manifest = {
  "audioSegments": [{"path": str(DRIVE_ASSETS / audio_file)}],
  "bgPath": str(DRIVE_ASSETS / bg_image),
  "config": {
    "aspectRatio": "9:16",
    "captions": {
      "fontSize": 92,
      "primaryColor": "#FFFFFF",
      "highlightColor": "#FFD700",
      "position": "Bottom"
    },
    "audio": {
      "whisperModel": "medium",
      "useVAD": True
    }
  }
}

path = pathlib.Path('/content/manifest.json')
path.write_text(json.dumps(manifest, ensure_ascii=False, indent=2))
print('✅ Manifest saved to', path)

In [ ]:
# 🎥 Submit render job and download output
import urllib.request

base_url = f"http://127.0.0.1:{os.environ['VR_PORT']}"
render_url = f"{base_url}/render"

with open('/content/manifest.json', 'r', encoding='utf-8') as f:
    manifest_data = json.load(f)

print(f"📤 Sending render request to {render_url}...")
resp = requests.post(render_url, json=manifest_data, timeout=60)
print('POST /render status:', resp.status_code)
print('Response:', resp.text)
resp.raise_for_status()

job_id = resp.json().get('jobId')
if not job_id:
    raise SystemExit('❌ No jobId returned from backend')

print(f'🎬 Render job started with ID: {job_id}')

# بررسی وضعیت رندر
status_url = f"{base_url}/status?jobId={job_id}"
for i in range(600): # حداکثر 10 دقیقه صبر
    r = requests.get(status_url, timeout=10)
    if r.ok:
        js = r.json()
        state = js.get('status')
        progress = js.get('progress', {})
        pct = progress.get('percentage', 0)
        msg = progress.get('message', '')
        print(f"Tick {i}: Status={state}, Progress={pct}%, Message='{msg}'")
        if state == 'done':
            print('✅ Render complete!')
            video_url = js.get('videoUrl')
            if video_url:
                video_name = video_url.split('?file=')[-1]
                download_url = f"{base_url}/download?file={video_name}"
                out_path = f"/content/{job_id}.mp4"
                print(f"📥 Downloading video from {download_url} to {out_path}...")
                urllib.request.urlretrieve(download_url, out_path)
                print(f'✅ Downloaded video: {out_path}')
                from google.colab import files
                files.download(out_path)
            else:
                print("❌ No video URL found in response.")
            break
        if state == 'error':
            error_msg = js.get('error', 'Unknown error')
            raise SystemExit(f'❌ Render job failed: {error_msg}')
    time.sleep(1)
else:
    raise SystemExit('❌ Render job did not finish in time')

print('🎉 Done! Check /content or your browser downloads for output.')